# Pre-processing after feat.eng.

In [60]:
import pandas as pd 

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.compose import make_column_selector as selector
from sklearn.model_selection import train_test_split

from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

import joblib

In [2]:
mode_imp = SimpleImputer(strategy='most_frequent')
scaler = StandardScaler()
encoder = OneHotEncoder(sparse=False,handle_unknown='ignore')

In [3]:
numeric_transform = Pipeline([('impute-mode', mode_imp),
                                ('scaling', scaler)])
categoric_transform = Pipeline([('impute-mode', mode_imp),
                                          ('one-hot-encode', encoder)])

In [4]:
pre_processing = ColumnTransformer([('numeric_cols',numeric_transform,selector(dtype_include="float64")),
                                    ('categoric_cols',categoric_transform,selector(dtype_include="object"))]
                                   ,remainder='drop')

In [5]:
X_train = pd.read_csv('train-test-1/X_train.csv')
y_train = pd.read_csv('train-test-1/y_train.csv')
X_test = pd.read_csv('train-test-1/X_test.csv')
y_test = pd.read_csv('train-test-1/y_test.csv')

In [6]:
def compare_model(model1,model2):
    y_pred1 = model1.predict(X_test)
    y_pred2 = model2.predict(X_test)
    
    r2_model1 = r2_score(y_test,y_pred1)
    r2_model2 = r2_score(y_test,y_pred2)
    
    if r2_model1 > r2_model2:
        return print('{} has better r2 score'.format(r2_model1))
    else:
        return print('{} has better r2 score'.format(r2_model2))

In [7]:
def progress_r2(model):
    y_pred=model.predict(X_test)
    r2 = r2_score(y_test,y_pred)
    return r2

In [8]:
r2_lst_2=[]
def lst_r2(r2):
    r2_lst_2.append(r2)

In [16]:
def progress_mae(model):
    y_pred=model.predict(X_test)
    mae = mean_absolute_error(y_test,y_pred)
    return mae

In [17]:
mae_lst=[]
def lst_mae(mae):
    mae_lst.append(mae)

# Support Vector Regressor

In [52]:
feature_union = FeatureUnion([('pca', PCA()), 
                              ('select_best', SelectKBest())])

svr_pipeline = Pipeline(steps=[('processing', pre_processing),
                               ('features',feature_union),
                           ('svr',SVR())])

In [53]:
%%time
# Find the best hyperparameters using GridSearchCV on the train set
param_grid = {'svr__kernel':['rbf', 'sigmoid'],
              'svr__C':[1,1.5,2,2.2,2.5,3],
              'features__pca__n_components':[4,5,6,7],
             'features__select_best__k':[4,5,6,7,8,9,10]
             }
svr_grid = GridSearchCV(svr_pipeline, param_grid=param_grid, cv=5)
svr_grid.fit(X_train, y_train)

/opt/anaconda3/envs/ML_env/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/envs/ML_env/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/opt/anaconda3/envs/ML_env/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/envs/ML_env/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/o

CPU times: user 30min 41s, sys: 1min 43s, total: 32min 24s
Wall time: 27min 31s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('processing',
                                        ColumnTransformer(transformers=[('numeric_cols',
                                                                         Pipeline(steps=[('impute-mode',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('scaling',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7fcad2944278>),
                                                                        ('categoric_cols',
                                                                         Pipeline(steps=[('impute-mode',
                                  

In [19]:
svr_pipeline.fit(X_train,y_train)

/opt/anaconda3/envs/ML_env/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Pipeline(steps=[('processing',
                 ColumnTransformer(transformers=[('numeric_cols',
                                                  Pipeline(steps=[('impute-mode',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fcad2944278>),
                                                 ('categoric_cols',
                                                  Pipeline(steps=[('impute-mode',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot-encode',
                                                                  

In [20]:
lst_mae(progress_mae(svr_pipeline))

In [58]:
r2_lst_2

[0.24045267511282498,
 0.31365686155465533,
 0.35656774242714573,
 0.32900594471921396]

In [54]:
best_svr = svr_grid.best_estimator_
best_hyperparams_svr = svr_grid.best_params_

In [55]:
lst_r2(progress_r2(best_svr))

In [56]:
lst_mae(progress_mae(best_svr))

In [57]:
mae_lst

[3.291751058990574, 3.115226700387827, 2.8713496328758414, 2.955848749963002]

In [35]:
best_hyperparams_svr

{'svr__C': 3, 'svr__kernel': 'rbf'}

In [29]:
y_pred = svr_grid.predict(X_test)

In [30]:
r2_score(y_test,y_pred)

0.31365686155465533

In [31]:
mean_absolute_error(y_test,y_pred)

3.115226700387827

In [32]:
mean_squared_error(y_test,y_pred)

48.52979562685479

In [33]:
joblib.dump(best_svr, 'models-1/baseline_svr.pkl')

['models-1/baseline_svr.pkl']

# Random Forest Regressor

In [198]:
pipe_RandForest = Pipeline(steps=[('processing', pre_processing),
                                  ('RForest',RandomForestRegressor())])

In [205]:
%%time
# Find the best hyperparameters using GridSearchCV on the train set
param_grid = {'RForest__n_estimators':[10,20,35,55,80,100],
             'RForest__min_samples_split':[4,6,8,10],
             'RForest__min_samples_leaf':[1,3,5,7,9],
             }
rForest_grid = GridSearchCV(pipe_RandForest, param_grid=param_grid, cv=5)
rForest_grid.fit(X_train, y_train)

CPU times: user 9min 47s, sys: 5.36 s, total: 9min 53s
Wall time: 10min 27s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('processing',
                                        ColumnTransformer(transformers=[('numeric_cols',
                                                                         Pipeline(steps=[('impute-mode',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('scaling',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7fa366907780>),
                                                                        ('categoric_cols',
                                                                         Pipeline(steps=[('impute-mode',
                                  

In [209]:
best_rForest = rForest_grid.best_estimator_
best_hyperparams_rForest = rForest_grid.best_params_

In [211]:
best_hyperparams_rForest

{'RForest__min_samples_leaf': 5,
 'RForest__min_samples_split': 8,
 'RForest__n_estimators': 35}

In [212]:
y_pred = rForest_grid.predict(X_test)

In [213]:
r2_score(y_test,y_pred)

0.4514790347792569

In [230]:
joblib.dump(best_rForest, 'models-1/rForest_v2.pkl')

['models-1/baseline_rForest.pkl']

# XGBRegressor

In [260]:
pipe_GrBoost = Pipeline(steps=[('processing', pre_processing),
                           ('GrBoost',GradientBoostingRegressor(loss='lad',learning_rate=0.1,n_estimators=100,max_depth=6,min_samples_split=5))])

In [265]:
pipe_GrBoost.fit(X_train,y_train)

Pipeline(steps=[('processing',
                 ColumnTransformer(transformers=[('numeric_cols',
                                                  Pipeline(steps=[('impute-mode',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa366907780>),
                                                 ('categoric_cols',
                                                  Pipeline(steps=[('impute-mode',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot-encode',
                                                                  

In [278]:
y_pred = pipe_GrBoost.predict(X_test)

In [279]:
r2_score(y_test,y_pred)

0.4418261732614325

In [273]:
joblib.dump(pipe_GrBoost, 'models-1/baseline_GrBoost.pkl')

['models-1/baseline_GrBoost.pkl']

# Elastic net

In [284]:
pipe_eNet = Pipeline(steps=[('processing', pre_processing),
                           ('eNet',ElasticNet())])

In [285]:
%%time
# Find the best hyperparameters using GridSearchCV on the train set
param_grid = {'eNet__alpha':[0.1,0.2,0.5,1],
             'eNet__l1_ratio':[0.1,0.5,0.7],
             'eNet__max_iter':[1000,2000]}
eNet_grid = GridSearchCV(pipe_eNet, param_grid=param_grid, cv=5)
eNet_grid.fit(X_train, y_train)

CPU times: user 17.4 s, sys: 3.27 s, total: 20.7 s
Wall time: 7.83 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('processing',
                                        ColumnTransformer(transformers=[('numeric_cols',
                                                                         Pipeline(steps=[('impute-mode',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('scaling',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7fa366907780>),
                                                                        ('categoric_cols',
                                                                         Pipeline(steps=[('impute-mode',
                                  

In [286]:
best_eNet = eNet_grid.best_estimator_
best_hyperparams_eNet = eNet_grid.best_params_

In [287]:
best_hyperparams_eNet

{'eNet__alpha': 0.1, 'eNet__l1_ratio': 0.1, 'eNet__max_iter': 1000}

In [288]:
y_pred = best_eNet.predict(X_test)
r2_score(y_test,y_pred)

0.3031358422738887

In [289]:
joblib.dump(best_eNet, 'models-1/baseline_eNet.pkl')

['models-1/baseline_eNet.pkl']